# Molecular atmosphere

In this tutorial, we show how to create a molecular atmosphere scene element and how to inspect the properties of the objects we create.

In [ ]:
# We load the Rich notebook extension for improved object inspection
%load_ext rich

# We import the top-level Eradiate module and the MolecularAtmosphere class
import eradiate
from eradiate.scenes.atmosphere import MolecularAtmosphere

# We'll use the correlated k-distribution mode
eradiate.set_mode("ckd")

## Getting started

Create molecular atmosphere objects by instanciating the [MolecularAtmosphere](../rst/reference_api/generated/autosummary/eradiate.scenes.atmosphere.MolecularAtmosphere.rst) class:

In [ ]:
my_atmosphere = MolecularAtmosphere()

Display the object:

In [ ]:
my_atmosphere

Let us focus here on the most basic attributes, namely ``thermoprops``, ``phase``, ``has_absorption`` and ``has_scattering``.

The ``thermoprops`` specify the [atmosphere thermophysical properties](../rst/user_guide/data/atmosphere_thermoprops.rst).
By default, the *U.S. Standard Atmosphere, 1976* thermophysical properties are used.

The ``phase`` attribute specifies the scattering phase function of the atmosphere.
By default, it is set to the Rayleigh phase function.

Finally the ``has_absorption`` and ``has_scattering`` attributes are like switches that enable or disable absorption or scattering, respectively, in the atmosphere.
By default, they are both set to ``True``.

This is the default molecular atmosphere.
In the next sections, we will show how to create other types of molecular atmospheres and customise them.

## Use pre-defined thermophysical properties and customise them

Eradiate comes with pre-defined thermophysical properties data sets.


### U.S. Standard Atmosphere, 1976

This is the default thermophysical properties.
A molecular atmosphere using the *U.S. Standard Atmosphere, 1976* thermophysical properties is created using:

In [ ]:
my_us76_atmosphere = MolecularAtmosphere.ussa_1976()

This creates the same molecular atmosphere as the default constructor `MolecularAtmosphere()`.
However, it lets us customise a little bit the thermophysical properties, as illustrated below:

In [ ]:
import numpy as np
from eradiate import unit_registry as ureg

my_custom_us76_atmosphere = MolecularAtmosphere.ussa_1976(
    levels=np.linspace(0, 32, 33) * ureg.km,
)

In the cell above, we have created an atmosphere with a modified version of the *U.S. Standard Atmosphere, 1976* thermophysical properties.

We have set the level altitude grid to a linearly spaced mesh from 0 to 32 km with 33 points, i.e., 32 atmospheric layers.

Let us inspect our customised atmosphere:

In [ ]:
my_custom_us76_atmosphere

As expected, our custom atmosphere has a height of 32 km:

In [ ]:
my_custom_us76_atmosphere.height.to("km")

### AFGL (1986) *reference atmospheres*

The *U.S. Standard Atmosphere, 1976* thermophysical properties are popular but not the most relevant for radiative transfer applications, especially in spectral regions where absorption plays an important role.
The six *reference atmospheres* published in the 1986 report *AFGL Atmospheric Constituent Profile (0-120km)* by Anderson et al are more appropriate to radiative transfer applications.

Let us create a molecular atmosphere based on the *AFGL (1986) - U.S. Standard* thermophysical properties:

In [ ]:
us_standard_atmosphere = MolecularAtmosphere.afgl_1986(model="us_standard")

We can choose another *reference atmosphere*, e.g. the *Midlatitude Summer reference atmosphere*, using the `model` parameter:

In [ ]:
midlatitude_summer_atmosphere = MolecularAtmosphere.afgl_1986(model="midlatitude_summer")

We can confirm that the *U.S. Standard Atmosphere, 1976* and *AFGL (1986) - U.S. Standard* thermophysical properties differ in particular with respect to molecular species present in the atmosphere:

In [ ]:
list(my_us76_atmosphere.thermoprops.species.values)

In [ ]:
list(us_standard_atmosphere.thermoprops.species.values)

The *U.S. Standard Atmosphere, 1976* focuses on abundant molecular species whereas the *AFGL (1986) - U.S. Standard* focuses on radiatively active molecular species.

We can customise the AFGL (1986) reference atmospheres using the `levels` and `concentrations` attributes.
The `levels` attribute works just the same way as with `MolecularAtmosphere.ussa_1976()`.
The `concentrations` attribute accepts a dictionary that maps molecular species and their target concentration in the atmosphere.

In the example below, we customise the *AFGL (1986) - U.S. Standard* thermophysical properties so that:

* the altitude grid is a linear mesh from 0 to 64 km, with 32 layers (2km-thick each). 
* the column mass density of water vapor is 15 kg / m^2,
* the volume mixing ratio of CO2 at sea level is 400 ppm,
* the column number density of O3 is 350 Dobson units:

In [ ]:
custom_us_standard_atmosphere = MolecularAtmosphere.afgl_1986(
    model="us_standard",
    levels=np.linspace(0, 64, 33) * ureg.km,
    concentrations={
        "H2O": 15.0 * ureg.kg / ureg.m ** 2,  # column mass density
        "CO2": 400e-6 * ureg.dimensionless,  # volume mixing fraction at sea level
        "O3": 350.0 * ureg.dobson_units,  # column number density
    }
)

## Enable or disable atmospheric scattering and/or absorption

In some situations, it is useful to be able to enable/disable atmospheric scattering or absorption.
This is easily achieved using the ``has_absorption`` and ``has_scattering`` attributes:

In [ ]:
non_absorbing_atmosphere = MolecularAtmosphere.afgl_1986(
    has_absorption=False,
    has_scattering=True,
)  # atmosphere scatters but does not absorb light

non_scattering_atmosphere = MolecularAtmosphere.afgl_1986(
    has_absorption=True,
    has_scattering=False,
)  # atmosphere absorbs but does not scatter light

Disabling both at the same time does not make much sense and will raise a ``ValueError``:

In [ ]:
MolecularAtmosphere.afgl_1986(
    has_scattering=False,
    has_absorption=False,
)